In [127]:
import time
import numpy as np
import numpy.random as rng
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [128]:
class SimTooLongException(Exception):

    def __init__(self, max_n_steps):
        self.max_n_steps = max_n_steps

    def __str__(self):
        return 'Simulation exceeded the maximum of {} steps.'.format(self.max_n_steps)


In [129]:
def discrete_sample(p, n_samples=1):
    """
    Samples from a discrete distribution.
    :param p: a distribution with N elements
    :param n_samples: number of samples
    :return: vector of samples
    """

    # check distribution
    #assert isdistribution(p), 'Probabilities must be non-negative and sum to one.'

    # cumulative distribution
    c = np.cumsum(p[:-1])[np.newaxis, :]

    # get the samples
    r = rng.rand(n_samples, 1)
    return np.sum((r > c).astype(int), axis=1)


In [130]:
class MarkovJumpProcess:
    """Implements a generic markov jump process and algorithms for simulating it.
    It is an abstract class, it needs to be inherited by a concrete implementation."""

    def __init__(self, init, params):

        self.state = np.asarray(init)
        self.params = np.asarray(params)
        self.time = 0.0

    def _calc_propensities(self):
        raise NotImplementedError('This is an abstract method and should be implemented in a subclass.')

    def _do_reaction(self, reaction):
        raise NotImplementedError('This is an abstract method and should be implemented in a subclass.')

    def sim_steps(self, num_steps):
        """Simulates the process with the gillespie algorithm for a specified number of steps."""

        times = [self.time]
        states = [self.state.copy()]

        for _ in range(num_steps):

            rates = self.params * self._calc_propensities()
            total_rate = rates.sum()

            if total_rate == 0:
                self.time = float('inf')
                break

            self.time += rng.exponential(scale=1/total_rate)

            reaction = discrete_sample(rates / total_rate)[0]
            self._do_reaction(reaction)

            times.append(self.time)
            states.append(self.state.copy())

        return times, np.array(states)

    def sim_time(self, dt, duration, max_n_steps=float('inf')):
        """Simulates the process with the gillespie algorithm for a specified time duration."""

        num_rec = int(duration / dt) + 1
        states = np.zeros([num_rec, self.state.size])
        cur_time = self.time
        n_steps = 0

        for i in range(num_rec):

            while cur_time > self.time:

                rates = self.params * self._calc_propensities()
                total_rate = rates.sum()

                if total_rate == 0:
                    self.time = float('inf')
                    break

                self.time += rng.exponential(scale=1/total_rate)

                reaction = discrete_sample(rates / total_rate)[0]
                self._do_reaction(reaction)

                n_steps += 1
                if n_steps > max_n_steps:
                    raise SimTooLongException(max_n_steps)

            states[i] = self.state.copy()
            cur_time += dt

        return np.array(states)

In [131]:
class LotkaVolterra(MarkovJumpProcess):
    """Implements the lotka-volterra population model."""

    def _calc_propensities(self):

        x, y = self.state
        xy = x * y
        return np.array([xy, x, y, xy])

    def _do_reaction(self, reaction):

        if reaction == 0:
            self.state[0] += 1
        elif reaction == 1:
            self.state[0] -= 1
        elif reaction == 2:
            self.state[1] += 1
        elif reaction == 3:
            self.state[1] -= 1
        else:
            raise ValueError('Unknown reaction.')

In [132]:
datadir = "/home/mauro/Documents/University/MCMCPlayground/data/lotka-volterra/"

In [133]:
def run_smc_abc():
    """Runs sequential monte carlo abc and saves results."""

    # set parameters
    n_particles = 100
    eps_init = 10.0
    eps_last = 0.1
    eps_decay = 0.9
    ess_min = 0.5

    # load pilot results and observed statistics
    pilot_means, pilot_stds = pd.read_pickle(datadir + 'pilot_run_results.pkl')  #load(datadir + 'pilot_run_results.pkl')

    obs_stats = pd.read_pickle(datadir + 'obs_stats.pkl') #load(datadir + 'obs_stats.pkl')
    obs_stats -= pilot_means
    obs_stats /= pilot_stds

    all_params = []
    all_logweights = []
    all_eps = []
    all_nsims = []
    n_dim = len(true_params)

    # sample initial population
    params = np.empty([n_particles, n_dim])
    weights = np.ones(n_particles, dtype=float) / n_particles
    logweights = np.log(weights)
    eps = eps_init
    iter = 0
    nsims = 0

    for i in range(n_particles):

        dist = float('inf')

        while dist > eps:
            params[i] = sim_prior_params()

            lv = LotkaVolterra(init, params[i])
            try:
                nsims += 1
                states = lv.sim_time(dt, duration, max_n_steps=max_n_steps)
            except SimTooLongException:
                continue

            stats = calc_summary_stats(states)
            stats -= pilot_means
            stats /= pilot_stds

            dist = calc_dist(stats, obs_stats)

        print('particle = {0}'.format(i))

    all_params.append(params)
    all_logweights.append(logweights)
    all_eps.append(eps)
    all_nsims.append(nsims)

    print('iteration = {0}, eps = {1:.2}, ess = {2:.2%}'.format(iter, eps, 1.0))

    while eps > eps_last:

        iter += 1
        eps *= eps_decay

        # calculate population covariance
        logparams = np.log(params)
        mean = np.mean(logparams, axis=0)
        cov = 2.0 * (np.dot(logparams.T, logparams) / n_particles - np.outer(mean, mean))
        std = np.linalg.cholesky(cov)

        # perturb particles
        new_params = np.empty_like(params)
        new_logweights = np.empty_like(logweights)

        for i in range(n_particles):

            dist = float('inf')

            while dist > eps:
                idx = discrete_sample(weights)[0]
                new_params[i] = params[idx] * np.exp(np.dot(std, rng.randn(n_dim)))

                lv = LotkaVolterra(init, new_params[i])
                try:
                    nsims += 1
                    states = lv.sim_time(dt, duration, max_n_steps=max_n_steps)
                except SimTooLongException:
                    continue

                stats = calc_summary_stats(states)
                stats -= pilot_means
                stats /= pilot_stds

                dist = calc_dist(stats, obs_stats)

            new_logparams_i = np.log(new_params[i])
            logkernel = -0.5 * np.sum(np.linalg.solve(std, (new_logparams_i - logparams).T) ** 2, axis=0)
            new_logweights[i] = -float('inf') if np.any(new_logparams_i > log_prior_max) or np.any(new_logparams_i < log_prior_min) else -scipy.misc.logsumexp(logweights + logkernel)

            print('particle = {0}'.format(i))

        params = new_params
        logweights = new_logweights - scipy.misc.logsumexp(new_logweights)
        weights = np.exp(logweights)

        # calculate effective sample size
        ess = 1.0 / (np.sum(weights ** 2) * n_particles)
        print('iteration = {0}, eps = {1:.2}, ess = {2:.2%}'.format(iter, eps, ess))

        if ess < ess_min:

            # resample particles
            new_params = np.empty_like(params)

            for i in range(n_particles):
                idx = discrete_sample(weights)[0]
                new_params[i] = params[idx]

            params = new_params
            weights = np.ones(n_particles, dtype=float) / n_particles
            logweights = np.log(weights)

        all_params.append(params)
        all_logweights.append(logweights)
        all_eps.append(eps)
        all_nsims.append(nsims)

        # save results
        filename = datadir + 'smc_abc_results.pkl'
        save((all_params, all_logweights, all_eps, all_nsims), filename)

In [134]:
def show_smc_abc_results():
    """
    Loads and shows the results from smc abc.
    """

    # read data
    all_params, all_logweights, all_eps, _ = pd.read_pickle(datadir + 'smc_abc_results.pkl') #load(datadir + 'smc_abc_results.pkl')
    n_dim = len(true_params)

    for params, logweights, eps in izip(all_params, all_logweights, all_eps):

        weights = np.exp(logweights)
        logparams = np.log(params)

        # print estimates with error bars
        means = np.dot(weights, logparams)
        stds = np.sqrt(np.dot(weights, logparams ** 2) - means ** 2)
        print('eps = {0:.2}'.format(eps))
        for i in range(n_dim):
            print('log theta {0}: true = {1:.2} \t estimate = {2:.2} +/- {3:.2}'.format(i+1, np.log(true_params[i]), means[i], 2.0 * stds[i]))
        print('')

        # plot histograms and scatter plots
        plot_hist_marginals(logparams, lims=[log_prior_min, log_prior_max], gt=np.log(true_params))
        plt.gcf().suptitle('tolerance = {0:.2}'.format(eps))

    plt.show(block=False)

In [135]:
def load(file):
    """Loads data from file."""

    f = open(file, 'r')
    data = pickle.load(f)
    f.close()
    return data

In [136]:
def save(data, file):
    """Saves data to a file."""

    f = open(file, 'w')
    pickle.dump(data, f)
    f.close()

In [137]:
def plot_hist_marginals(data, lims=None, gt=None):
    # Plots marginal histograms and pairwise scatter plots of a dataset.

    n_bins = int(np.sqrt(data.shape[0]))

    if data.ndim == 1:

        fig, ax = plt.subplots(1, 1)
        ax.hist(data, n_bins, normed=True)
        ax.set_ylim([0, ax.get_ylim()[1]])
        if lims is not None: ax.set_xlim(lims)
        if gt is not None: ax.vlines(gt, 0, ax.get_ylim()[1], color='r')

    else:

        n_dim = data.shape[1]
        fig, ax = plt.subplots(n_dim, n_dim)
        ax = np.array([[ax]]) if n_dim == 1 else ax

        if lims is not None:
            lims = np.asarray(lims)
            lims = np.tile(lims, [n_dim, 1]) if lims.ndim == 1 else lims

        for i in range(n_dim):
            for j in range(n_dim):

                if i == j:
                    ax[i, j].hist(data[:, i], n_bins, normed=True)
                    ax[i, j].set_ylim([0, ax[i, j].get_ylim()[1]])
                    if lims is not None: ax[i, j].set_xlim(lims[i])
                    if gt is not None: ax[i, j].vlines(gt[i], 0, ax[i, j].get_ylim()[1], color='r')

                else:
                    ax[i, j].plot(data[:, i], data[:, j], 'k.', ms=2)
                    if lims is not None:
                        ax[i, j].set_xlim(lims[i])
                        ax[i, j].set_ylim(lims[j])
                    if gt is not None: ax[i, j].plot(gt[i], gt[j], 'r.', ms=8)

    plt.show(block=False)

    return fig, ax


In [138]:
init = [50, 100]
dt = 0.2
duration = 30
true_params = [0.01, 0.5, 1.0, 0.01]
log_prior_min = -5
log_prior_max = 2
max_n_steps = 10000

In [140]:
def sim_prior_params(num_sims=1):
    """
    Simulates parameters from the prior. Assumes a uniform prior in the log domain.
    """

    z = rng.rand(4) if num_sims == 1 else rng.rand(num_sims, 4)
    return np.exp((log_prior_max - log_prior_min) * z + log_prior_min)


In [142]:
def calc_summary_stats(states):
    """
    Given a sequence of states produced by a simulation, calculates and returns a vector of summary statistics.
    Assumes that the sequence of states is uniformly sampled in time.
    """

    N = states.shape[0]
    x, y = states[:, 0].copy(), states[:, 1].copy()

    # means
    mx = np.mean(x)
    my = np.mean(y)

    # variances
    s2x = np.var(x, ddof=1)
    s2y = np.var(y, ddof=1)

    # standardize
    x = (x - mx) / np.sqrt(s2x)
    y = (y - my) / np.sqrt(s2y)

    # auto correlation coefficient
    acx = []
    acy = []
    for lag in [1, 2]:
        acx.append(np.dot(x[:-lag], x[lag:]) / (N-1))
        acy.append(np.dot(y[:-lag], y[lag:]) / (N-1))

    # cross correlation coefficient
    ccxy = np.dot(x, y) / (N-1)

    return np.array([mx, my, np.log(s2x + 1), np.log(s2y + 1)] + acx + acy + [ccxy])



In [144]:
def calc_dist(stats_1, stats_2):
    """
    Calculates the distance between two vectors of summary statistics. Here the euclidean distance is used.
    """

    return np.sqrt(np.sum((stats_1 - stats_2) ** 2))


In [145]:
run_smc_abc()

KeyboardInterrupt: 